## RUM Analysis for Pricing optimization

***This notebook is to perform all operations related to the RUM analysis for all three Pricing Optimization runs:***
* MES lift optimization
* Units lift optimization
* Multi-Objective optimization with equal weights for MES and Units lift

P.S: the analysis here is done for only one CBSA

In [0]:
import pandas as pd
import os
import numpy as np
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import Window

import pickle
import random

In [0]:
owner = dbutils.widgets.get("owner")
cbsa = dbutils.widgets.get("cbsa")
budget = dbutils.widgets.get("budget")
budget = str(budget)[:4].replace('.', '_')
target_cbsa = dbutils.widgets.get("target_cbsa")
train_date = dbutils.widgets.get("train_date")
zone_id = dbutils.widgets.get("zone_id")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1159622303410155> in <module> 
 ----> 1 owner = dbutils . widgets . get ( "owner" ) 
 2 cbsa = dbutils . widgets . get ( "cbsa" ) 
 3 budget = dbutils . widgets . get ( "budget" ) 
 4 budget = str ( budget ) [ : 4 ] . replace ( '.' , '_' ) 
 5 target_cbsa = dbutils . widgets . get ( "target_cbsa" ) 

 /local_disk0/tmp/1606931566553-0/WidgetsHandlerImpl.py in get (self, name) 
 42 : return : Current value of the widget or default value
 43 """
 ---> 44 return self . _notebookArguments . getArgument ( name , self . _entry_point . getCurrentBindings ( ) ) 
 45 
 46 def getArgument ( self , name , defaultValue = None ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o220.getArgument.
: com.databricks.dbutils_v1.InputWidgetNotDefined: No input widget named owner is defined
	at com.databricks.backend.daemon.driver.NotebookArguments.checkExists(NotebookArguments.scala:72)
	at com.databricks.backend.daemon.driver.NotebookArguments.getArgument(NotebookArguments.scala:258)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
dbname = owner + '_' + cbsa
prefix = dbname + '.opti_result_rum_'
mes_file = dbname + '.cbsa_mes_comp_zone' + zone_id
units_elasticity_file = dbname + '.lp_opti_input_filtered_zone' + zone_id
all_items_file = dbname + '.RUM_all_item_compsub'

In [0]:
# load competition price table
comp_price_df = spark.read.table(dbname + '.comp_prices_wmt_cos_bj' + train_date)\
      .withColumnRenamed('system_item_nbr', 'scan_id')\
      .filter(F.col('costco_buying_zone') == target_cbsa)\
      .drop(*['costco_buying_zone'])

In [0]:
# load the table containing all items in the clubs
df_all_items = spark.read.table(all_items_file)\
      .join(comp_price_df, on='scan_id', how='left')\
      .withColumn('WMT_Price_Gap_before', F.round(F.col('Sell_Price_used_in_Optimization') - F.col('wmt_price'),2))\
      .withColumn('COS_Price_Gap_before', F.round(F.col('Sell_Price_used_in_Optimization') - F.col('cos_price'),2))\
      .withColumn('BJ_Price_Gap_before', F.round(F.col('Sell_Price_used_in_Optimization') - F.col('bj_price'),2))\
      .where('superzone=={}'.format(zone_id))\
      .withColumnRenamed('Sell_Price_used_in_Optimization', 'retail')\
      .withColumnRenamed('Cost_used_in_Optimization', 'warehouse_pack_cost')

# display(df_all_items.filter("scan_id = 32503851"))      

In [0]:
display(df_all_items.select('Item_Scope_Flag').groupBy(F.col('Item_Scope_Flag')).count())

In [0]:
# load the table containing MES data
# this is used to get the actual and predicted MES values
# at the price point picked by the optimization
df_mes = spark.read.table(mes_file)\
      .withColumnRenamed('caring_item', 'scan_id_mes')\
      .selectExpr(['scan_id_mes', 'price_point as price_points_elas', 'price_point_cbsa_mes', 'selling_price_cbsa_mes']).distinct()
# display(df_mes.filter("scan_id_mes = 32503851"))

In [0]:
# load results for all 3 types of optimization performed
results_MOO_init = spark.read.table(prefix + '50MES_MOO_lift_' + budget + 'M_zone' + zone_id)
results_MES_init = spark.read.table(prefix + 'MES_lift_' + budget + 'M_zone' + zone_id)
results_Units_init = spark.read.table(prefix + 'Units_lift_' + budget + 'M_zone' + zone_id)

In [0]:
display(results_MOO_init.select('y1').groupBy('y1').count())

In [0]:
mes_items = results_MES_init.select('scan_id')
moo_items = results_MOO_init.select('scan_id')
units_items = results_Units_init.select('scan_id')

items_df = mes_items.union(moo_items)\
      .union(units_items).distinct()\
      .withColumn('optimization_flag', F.lit('Optimization_picked'))

df_all_items_flagged = df_all_items\
      .join(items_df, on='scan_id', how='left')\
      .withColumn('Item_status_flag', F.coalesce('optimization_flag', 'Item_Scope_Flag'))\
      
# display(df_all_items_flagged.filter('scan_id = 15151'))

In [0]:
# MES results preprocessing 
results_MES = results_MES_init\
      .selectExpr(['CBSA as CBSA_MES', 'scan_id',
                  'recommended_price as recommended_price_MES', 'MES as mes_diff_MES', 
                  'predicted_units as predicted_units_MES', 'predicted_profit as predicted_profit_MES', 'predicted_revenue as predicted_revenue_MES',
                  'predicted_biannual_units as predicted_biannual_units_MES', 'predicted_biannual_profit as predicted_biannual_profit_MES', 'predicted_biannual_revenue as predicted_biannual_revenue_MES',
                  'units_diff as units_diff_MES',
                  'y1 as Optimization_Flag_MES'])\
      .withColumn('mes_diff_MES', F.round('mes_diff_MES', 2))\
      .withColumn('units_lift_percent_MES', F.round((F.col('units_diff_MES')/F.col('predicted_biannual_units_MES')), 4))\

print(results_MES.count())
joind_cond_mes = [results_MES.scan_id == df_mes.scan_id_mes,
                 results_MES.recommended_price_MES == df_mes.price_points_elas]

results_MES = results_MES.join(df_mes, on=joind_cond_mes, how='left')\
      .join(df_all_items_flagged, on='scan_id', how='inner')\
      .withColumn('retail', F.round(F.col('retail'), 2))\
      .withColumn('price_diff_MES', F.round(F.col('price_points_elas')-F.col('retail'),2))\
      .withColumn('price_diff_percent_lift_MES', F.round(F.col('price_diff_MES')/F.col('retail'),4))\
      .withColumnRenamed('price_point_cbsa_mes', 'predicted_mes_MES')\
      .withColumnRenamed('selling_price_cbsa_mes', 'current_mes_MES')\
      .withColumn('mes_percent_lift_MES', F.round(F.col('mes_diff_MES')/F.col('current_mes_MES'), 4))\
      .withColumnRenamed('Total_Units_Sold', 'actual_biannual_units_MES')\
      .withColumnRenamed('actual_biannual_revenue', 'actual_biannual_revenue_MES')\
      .withColumnRenamed('actual_biannual_profit', 'actual_biannual_profit_MES',)\
      .withColumn('revenue_diff_MES', F.round(F.col('predicted_revenue_MES')-F.col('predicted_biannual_revenue_MES'),2))\
      .withColumn('revenue_lift_percent_MES', F.round(F.col('revenue_diff_MES')/F.col('predicted_biannual_revenue_MES'), 4))\
      .withColumn('profit_diff_MES', F.round(F.col('predicted_profit_MES')-F.col('predicted_biannual_profit_MES'), 2))\
      .withColumn('profit_lift_percent_MES', F.round(F.col('profit_diff_MES')/F.col('predicted_biannual_profit_MES'), 4))\
      .drop(*['scan_id_mes', 'price_point', 'scan_id_elas', 'price_points_elas', 'cbsa'])\
      .selectExpr(['Optimization_Flag_MES', 'CBSA_MES', 'scan_id', 'category_nbr', 
                   'warehouse_pack_cost as warehouse_pack_cost_MES', 
                   'retail as retail_MES', 
                   'recommended_price_MES', 'price_diff_MES', 'price_diff_percent_lift_MES',
                   'WMT_Price_Gap_before as WMT_Price_Gap_before_MES', '(recommended_price_MES - wmt_price) as WMT_Price_Gap_after_MES',
                   'COS_Price_Gap_before as COS_Price_Gap_before_MES', '(recommended_price_MES - cos_price) as COS_Price_Gap_after_MES',
                   'BJ_Price_Gap_before as BJ_Price_Gap_before_MES', '(recommended_price_MES - bj_price) as BJ_Price_Gap_after_MES',
                   'predicted_biannual_units_MES', 'predicted_units_MES', 'units_diff_MES', 'units_lift_percent_MES',
                   'current_mes_MES', 'predicted_mes_MES', 'mes_diff_MES', 'mes_percent_lift_MES',
                   'predicted_biannual_profit_MES', 'predicted_profit_MES', 'profit_diff_MES', 'profit_lift_percent_MES',
                   'predicted_biannual_revenue_MES', 'predicted_revenue_MES', 'revenue_diff_MES', 'revenue_lift_percent_MES',
                   'actual_biannual_units_MES', 'actual_biannual_revenue_MES', 'actual_biannual_profit_MES'
                  ])

# display(results_MES.filter('scan_id = 11857 or scan_id = 15151'))

In [0]:
# MOO results preprocessing 
results_MOO = results_MOO_init\
      .selectExpr(['CBSA as CBSA_MOO', 'scan_id',
                  'recommended_price as recommended_price_MOO', 'MES as mes_diff_MOO', 
                  'predicted_units as predicted_units_MOO', 'predicted_profit as predicted_profit_MOO', 'predicted_revenue as predicted_revenue_MOO',
                  'predicted_biannual_units as predicted_biannual_units_MOO', 'predicted_biannual_profit as predicted_biannual_profit_MOO', 'predicted_biannual_revenue as predicted_biannual_revenue_MOO',
                  'units_diff as units_diff_MOO',
                  'y1 as Optimization_Flag_MOO'])\
      .withColumn('mes_diff_MOO', F.round('mes_diff_MOO', 2))\
      .withColumn('units_lift_percent_MOO', F.round((F.col('units_diff_MOO')/F.col('predicted_biannual_units_MOO')), 4))\


joind_cond_mes = [results_MOO.scan_id == df_mes.scan_id_mes,
                  results_MOO.recommended_price_MOO == df_mes.price_points_elas]


results_MOO = results_MOO.join(df_mes, on=joind_cond_mes, how='left')\
      .join(df_all_items_flagged, on='scan_id', how='inner')\
      .withColumn('retail', F.round(F.col('retail'), 2))\
      .withColumn('price_diff_MOO', F.round(F.col('price_points_elas')-F.col('retail'),2))\
      .withColumn('price_diff_percent_lift_MOO', F.round(F.col('price_diff_MOO')/F.col('retail'),4))\
      .withColumnRenamed('price_point_cbsa_mes', 'predicted_mes_MOO')\
      .withColumnRenamed('selling_price_cbsa_mes', 'current_mes_MOO')\
      .withColumnRenamed('Total_Units_Sold', 'actual_biannual_units_MOO')\
      .withColumnRenamed('actual_biannual_revenue', 'actual_biannual_revenue_MOO')\
      .withColumnRenamed('actual_biannual_profit', 'actual_biannual_profit_MOO',)\
      .withColumn('mes_percent_lift_MOO', F.round(F.col('mes_diff_MOO')/F.col('current_mes_MOO'), 4))\
      .withColumn('revenue_diff_MOO', F.round(F.col('predicted_revenue_MOO')-F.col('predicted_biannual_revenue_MOO'),2))\
      .withColumn('revenue_lift_percent_MOO', F.round(F.col('revenue_diff_MOO')/F.col('predicted_biannual_revenue_MOO'), 4))\
      .withColumn('profit_diff_MOO', F.round(F.col('predicted_profit_MOO')-F.col('predicted_biannual_profit_MOO'), 2))\
      .withColumn('profit_lift_percent_MOO', F.round(F.col('profit_diff_MOO')/F.col('predicted_biannual_profit_MOO'), 4))\
      .drop(*['scan_id_mes', 'price_point', 'scan_id_elas', 'price_points_elas', 'cbsa'])\
      .selectExpr(['Optimization_Flag_MOO', 'CBSA_MOO', 'scan_id', 'category_nbr', 
                   'warehouse_pack_cost as warehouse_pack_cost_MOO', 
                   'retail as retail_MOO', 
                   'recommended_price_MOO', 'price_diff_MOO', 'price_diff_percent_lift_MOO',
                   'WMT_Price_Gap_before as WMT_Price_Gap_before_MOO', '(recommended_price_MOO - wmt_price) as WMT_Price_Gap_after_MOO',
                   'COS_Price_Gap_before as COS_Price_Gap_before_MOO', '(recommended_price_MOO - cos_price) as COS_Price_Gap_after_MOO',
                   'BJ_Price_Gap_before as BJ_Price_Gap_before_MOO', '(recommended_price_MOO - bj_price) as BJ_Price_Gap_after_MOO',
                   'predicted_biannual_units_MOO', 'predicted_units_MOO', 'units_diff_MOO', 'units_lift_percent_MOO',
                   'current_mes_MOO', 'predicted_mes_MOO', 'mes_diff_MOO', 'mes_percent_lift_MOO',
                   'predicted_biannual_profit_MOO', 'predicted_profit_MOO', 'profit_diff_MOO', 'profit_lift_percent_MOO',
                   'predicted_biannual_revenue_MOO', 'predicted_revenue_MOO', 'revenue_diff_MOO', 'revenue_lift_percent_MOO',
                   'actual_biannual_units_MOO', 'actual_biannual_revenue_MOO', 'actual_biannual_profit_MOO'       
                  ])

# display(results_MOO)

In [0]:
# Units results preprocessing 
results_Units = results_Units_init\
      .selectExpr(['CBSA as CBSA_Units', 'scan_id',
                  'recommended_price as recommended_price_Units', 'MES as mes_diff_Units', 
                  'predicted_units as predicted_units_Units', 'predicted_profit as predicted_profit_Units', 'predicted_revenue as predicted_revenue_Units',
                  'predicted_biannual_units as predicted_biannual_units_Units', 'predicted_biannual_profit as predicted_biannual_profit_Units', 'predicted_biannual_revenue as predicted_biannual_revenue_Units',
                   'units_diff as units_diff_Units',
                   'y1 as Optimization_Flag_Units'])\
      .withColumn('mes_diff_Units', F.round('mes_diff_Units', 2))\
      .withColumn('units_lift_percent_Units', F.round((F.col('units_diff_Units')/F.col('predicted_biannual_units_Units')), 4))\


joind_cond_mes = [results_Units.scan_id == df_mes.scan_id_mes,
                  results_Units.recommended_price_Units == df_mes.price_points_elas]

results_Units = results_Units.join(df_mes, on=joind_cond_mes, how='left')\
      .join(df_all_items_flagged, on='scan_id', how='inner')\
      .withColumn('retail', F.round(F.col('retail'), 2))\
      .withColumn('price_diff_Units', F.round(F.col('price_points_elas')-F.col('retail'),2))\
      .withColumn('price_diff_percent_lift_Units', F.round(F.col('price_diff_Units')/F.col('retail'),4))\
      .withColumnRenamed('price_point_cbsa_mes', 'predicted_mes_Units')\
      .withColumnRenamed('selling_price_cbsa_mes', 'current_mes_Units')\
      .withColumnRenamed('Total_Units_Sold', 'actual_biannual_units_Units')\
      .withColumnRenamed('actual_biannual_revenue', 'actual_biannual_revenue_Units')\
      .withColumnRenamed('actual_biannual_profit', 'actual_biannual_profit_Units',)\
      .withColumn('mes_percent_lift_Units', F.round(F.col('mes_diff_Units')/F.col('current_mes_Units'), 4))\
      .withColumn('revenue_diff_Units', F.round(F.col('predicted_revenue_Units')-F.col('predicted_biannual_revenue_Units'),2))\
      .withColumn('revenue_lift_percent_Units', F.round(F.col('revenue_diff_Units')/F.col('predicted_biannual_revenue_Units'), 4))\
      .withColumn('profit_diff_Units', F.round(F.col('predicted_profit_Units')-F.col('predicted_biannual_profit_Units'), 2))\
      .withColumn('profit_lift_percent_Units', F.round(F.col('profit_diff_Units')/F.col('predicted_biannual_profit_Units'), 4))\
      .drop(*['scan_id_mes', 'price_point', 'scan_id_elas', 'price_points_elas', 'cbsa'])\
      .selectExpr(['Optimization_Flag_Units', 'CBSA_Units', 'scan_id', 'category_nbr', 
                   'warehouse_pack_cost as warehouse_pack_cost_Units', 
                   'retail as retail_Units', 
                   'recommended_price_Units', 'price_diff_Units', 'price_diff_percent_lift_Units',
                   'WMT_Price_Gap_before as WMT_Price_Gap_before_Units', '(recommended_price_Units - wmt_price) as WMT_Price_Gap_after_Units',
                   'COS_Price_Gap_before as COS_Price_Gap_before_Units', '(recommended_price_Units - cos_price) as COS_Price_Gap_after_Units',
                   'BJ_Price_Gap_before as BJ_Price_Gap_before_Units', '(recommended_price_Units - bj_price) as BJ_Price_Gap_after_Units',
                   'predicted_biannual_units_Units', 'predicted_units_Units', 'units_diff_Units', 'units_lift_percent_Units',
                   'current_mes_Units', 'predicted_mes_Units', 'mes_diff_Units', 'mes_percent_lift_Units',
                   'predicted_biannual_profit_Units', 'predicted_profit_Units', 'profit_diff_Units', 'profit_lift_percent_Units',
                   'predicted_biannual_revenue_Units', 'predicted_revenue_Units', 'revenue_diff_Units', 'revenue_lift_percent_Units',
                   'actual_biannual_units_Units', 'actual_biannual_revenue_Units', 'actual_biannual_profit_Units'       
                  ])

# display(results_Units)

In [0]:
# combining all results in one table 
results_all = results_MOO\
      .join(results_MES, on=['scan_id', 'category_nbr'], how='outer')\
      .join(results_Units, on=['scan_id', 'category_nbr'], how='outer')\
      .drop(*['CBSA_MES', 'CBSA_Units'])\
      .withColumnRenamed('CBSA_MOO', 'CBSA')
# display(results_all)

In [0]:
df_all_items_details = df_all_items_flagged\
      .join(results_all, on=['scan_id', 'category_nbr'], how='left')\
      .drop(*['Item_Scope_Flag', 'optimization_flag'])\
      .na.fill({'cbsa': cbsa})
print(df_all_items_details.select('scan_id').count())

In [0]:
display(df_all_items_details)

In [0]:
# Load the file contianing a mapping between GMM, DMM and categories numbers
file_location = "/FileStore/tables/GMM_DMM_Category_map.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
GMM_DMM_Category_map = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)\
  .withColumnRenamed('cat_nbr', 'category_nbr')

In [0]:
# loading the item info table to use for details about items picked by the optimization models
# joining the item info table with DMM and GMM to map to categories description
item_info_df = spark.read.table('sams_us_clubs.item_info')\
      .join(GMM_DMM_Category_map, on='category_nbr', how='inner')\
      .selectExpr(["GMM", "DMM", 'system_item_nbr as scan_id', 'customer_item_nbr',
                   'item1_desc', 'item2_desc',
                   'category_nbr', 'category_desc', 
                   'sub_category_nbr', 'sub_category_desc'])\
      .withColumn('mm_item1_desc', F.trim(F.lower(F.col('item1_desc'))))\
      .withColumn('Members_Mark_Flag', 
                  F.when(F.col('mm_item1_desc').startswith('mm '), "Yes")
                   .when(F.col('mm_item1_desc').contains('membermark'), "Yes")
                   .when(F.col('mm_item1_desc').contains('membersmark'), "Yes")
                  .otherwise("No")
                 )\
      .distinct()

In [0]:
#loading info from scan table for the year 2019
# df_sales_total_init = spark.read.table('sams_us_clubs.scan')\
#       .filter(F.col('store_nbr').isin(club_ls))\
#       .filter(F.col('scan_type') == 0)\
#       .filter(F.col('retail_price') > 0)\
#       .filter(F.col('unit_qty') > 0)\
#       .withColumnRenamed('store_nbr', 'club_nbr')

# #filter for desired dates, and desired clubs with respect to CBSA
# df_sales_total = df_sales_total_init\
#       .filter((df_sales_total_init['visit_date']>='2019-01-01') & (df_sales_total_init['visit_date'] <= '2019-12-31'))

# df_sales_total = df_sales_total\
#       .join(item_info_df, on='scan_id', how='inner')\
#       .withColumn('cost_value', F.round(F.col('unit_qty') * F.col('unit_cost'),2))\
  
# df_units_cost = df_sales_total\
#       .groupBy(['category_nbr',
#                 'scan_id']
#                )\
#       .agg(F.sum('cost_value').alias('total_cost_value'))\
#       .drop('category_nbr')

In [0]:
# adding items infromation to the optimization results
results_item_info_all = df_all_items_details.join(item_info_df, on=['scan_id', 'category_nbr'], how='inner')\
      .selectExpr(['CBSA', "GMM", "DMM", 
                   'category_nbr as Category_Number', 'category_desc as Category', 
                   'sub_category_nbr as Sub_Cat_Number', 'sub_category_desc as Sub_Cat',
                   'scan_id', 'customer_item_nbr as Item_Number', 'item1_desc as Item_Description', 'item2_desc as Item_Description_2', 'Members_Mark_Flag', 
                   
                   'wmt_price as WMT_Price', 'WMT_Price_Gap_before', 
                   'cos_price as Costco_Price', 'COS_Price_Gap_before',
                   'bj_price as BJ_Price', 'BJ_Price_Gap_before',
                   
                   'Total_Units_Sold', 'actual_biannual_profit', 'actual_biannual_revenue',
                   'Total_Cost', 'Item_status_flag',
                   
                   'Optimization_Flag_MOO', 'warehouse_pack_cost_MOO as Cost_used_in_Optimization_MOO', 
                   'retail_MOO as Sell_Price_used_in_Optimization_MOO', 
                   '(retail_MOO - warehouse_pack_cost_MOO)/retail_MOO as current_margin_rate_MOO',
                   'recommended_price_MOO', 'price_diff_MOO as Price_change_MOO', 'price_diff_percent_lift_MOO as Price_Change_Percent_MOO',
                   '(recommended_price_MOO - warehouse_pack_cost_MOO)/recommended_price_MOO as new_margin_rate_MOO',
                   'WMT_Price_Gap_after_MOO',
                   'COS_Price_Gap_after_MOO',
                   'BJ_Price_Gap_after_MOO',
                   
                   'predicted_biannual_units_MOO as Pred_Biannual_Units_Sold_MOO', 'predicted_units_MOO as Pred_units_for_recommended_price_MOO', 
                   'units_diff_MOO as Pred_Volume_Lift_MOO', 'units_lift_percent_MOO as Pred_Volume_Lift_Percent_MOO',
                   
                   'current_mes_MOO', 
                   'predicted_mes_MOO', 'mes_diff_MOO', 'mes_percent_lift_MOO',
                   
                   'predicted_biannual_profit_MOO', 'predicted_profit_MOO as predicted_profit_for_recommended_price_MOO', 
                   'profit_diff_MOO', 'profit_lift_percent_MOO',
                   
                   'predicted_biannual_revenue_MOO', 'predicted_revenue_MOO as predicted_revenue_for_recommended_price_MOO', 
                   'revenue_diff_MOO', 'revenue_lift_percent_MOO',
                   
                   'Optimization_Flag_Units', 'warehouse_pack_cost_Units as Cost_used_in_Optimization_Units', 
                   'retail_Units as Sell_Price_used_in_Optimization_Units', 
                   '(retail_Units - warehouse_pack_cost_Units)/retail_Units as current_margin_rate_Units',
                   'recommended_price_Units', 'price_diff_Units as Price_change_Units', 'price_diff_percent_lift_Units as Price_Change_Percent_Units',
                   '(recommended_price_Units - warehouse_pack_cost_Units)/recommended_price_Units as new_margin_rate_Units',
                   'WMT_Price_Gap_after_Units',
                   'COS_Price_Gap_after_Units',
                   'BJ_Price_Gap_after_Units',
                   
                   'predicted_biannual_units_Units as Pred_Biannual_Units_Sold_Units', 'predicted_units_Units as Pred_units_for_recommended_price_Units', 
                   'units_diff_Units as Pred_Volume_Lift_Units', 'units_lift_percent_Units as Pred_Volume_Lift_Percent_Units',
                   
                   'current_mes_Units', 
                   'predicted_mes_Units', 'mes_diff_Units', 'mes_percent_lift_Units',
                   
                   'predicted_biannual_profit_Units', 'predicted_profit_Units as predicted_profit_for_recommended_price_Units', 
                   'profit_diff_Units', 'profit_lift_percent_Units',
                   
                   'predicted_biannual_revenue_Units', 'predicted_revenue_Units as predicted_revenue_for_recommended_price_Units', 
                   'revenue_diff_Units', 'revenue_lift_percent_Units',
                   
                   'Optimization_Flag_MES', 'warehouse_pack_cost_MES as Cost_used_in_Optimization_MES', 
                   'retail_MES as Sell_Price_used_in_Optimization_MES', 
                   '(retail_MES - warehouse_pack_cost_MES)/retail_MES as current_margin_rate_MES',
                   'recommended_price_MES', 'price_diff_MES as Price_change_MES', 'price_diff_percent_lift_MES as Price_Change_Percent_MES',
                   '(recommended_price_MES - warehouse_pack_cost_MES)/recommended_price_MES as new_margin_rate_MES',
                   'WMT_Price_Gap_after_MES',
                   'COS_Price_Gap_after_MES',
                   'BJ_Price_Gap_after_MES',
                   
                   'predicted_biannual_units_MES as Pred_Biannual_Units_Sold_MES', 'predicted_units_MES as Pred_units_for_recommended_price_MES', 
                   'units_diff_MES as Pred_Volume_Lift_MES', 'units_lift_percent_MES as Pred_Volume_Lift_Percent_MES',
                   
                   'current_mes_MES', 
                   'predicted_mes_MES', 'mes_diff_MES', 'mes_percent_lift_MES',
                   
                   'predicted_biannual_profit_MES', 'predicted_profit_MES as predicted_profit_for_recommended_price_MES', 
                   'profit_diff_MES', 'profit_lift_percent_MES',
                   
                   'predicted_biannual_revenue_MES', 'predicted_revenue_MES as predicted_revenue_for_recommended_price_MES', 
                   'revenue_diff_MES', 'revenue_lift_percent_MES',
                   
              ])\
      .withColumn('CBSA', F.lit('zone_' + zone_id)) \
      .drop_duplicates(['scan_id', 'Category_Number'])
# display(results_item_info_all)
#      .fillna('zone' + zone_id, subset='CBSA')\

In [0]:
results_item_info_all\
    .write.format('delta').mode('overwrite').option('overwriteSchema', 'true')\
    .saveAsTable(dbname + '.Summary_RUM_analysis_MES_MOO_Units_' + budget + 'M_budget_zone' + zone_id)

In [0]:
#print(prefix + 'analysis_LP_' + budget + 'M_budget')

In [0]:
#results_item_info_all = spark.read.table(prefix + 'analysis_LP_' + budget + 'M_budget')
#display(results_item_info_all)

In [0]:
#print(results_item_info_all.select('scan_id').count())

## Summarized RUM analysis

This is to calculate the total RUM lift for all three optimization models. Calculations performed are:
1. Number of items recommended for price investment for each model
2. Units lift --> Total units predicted to be sold at current price - Total number of units predicted to be sold at recommended price 
3. Units lift % --> Units lift / Total units sold in year 2019 for all items in PI categories 
4. Profit lift $ --> recommended_price * units_lift - unit_cost * units lift
5. Profit lift % --> This consists of two parts:
   1. Total Profit for all items in PI categories = selling_price * unit_qty -  unit_cost * unit_qty (this is retrieved from the scan table)
   2. Total Profit for recommended items =  recommended_price * units_lift - unit_cost * units lift
   3. Profit of recommended items / Profit of all PI items * 100
6. Revenue lift $ --> recommended_price * units_lift  
7. Revenue lift % -->
   1. Total revenue of all items in PI category = selling_price * unit_qty (this is retrieved from the scan table)
   2. Total revenue for recommended items = recommended_price * units_lift
   3. Revenue of recommended items / REvenue of all PI items * 100
8. MES lift -->
9. Max price drop --> get the difference in price for all items, and retreive the max of that price drop
10. Max price drop % --> get the max percentage of the price drop with respect to the current selling price (filter out outliers with more that 30% price drop)
11. Min price drop --> get the difference in price for all items, and retreive the min of that price drop
12. Min price Drop % --> get the min percentage of the price drop with respect to the current selling price
13. Average price drop --> get the difference in price for all items, and retreive the average
14. Average price drop % --> get the average percentage of the price drop with respect to the current selling price

In [0]:
# initializing a dictionary for RUM Summary Analysis
RUM_summary_pd = {'Opt_Model': [0,1,2],
                  'Items_Recommended': [],
                  'Current_units_volume': [], 'Recommended_units_volume': [], 'Units_lift': [], 
                  'Units_lift_Percent_Recommended_Items': [], 'Units_lift_Percent_Category_Level': [], 'Units_lift_Percent_Box_Level': [],
                  
                  'Pred_total_sales': [], 'Recommended_total_sales': [], 
                  'Total_sales_lift': [], 
                  'Total_sales_lift_percent_Recommended_Items': [], 'Total_sales_lift_percent_Category_Level': [], 'Total_sales_lift_percent_Box_Level': [],

                  'Pred_total_margin': [], 'Recommended_total_margin': [], 
                  'Total_margin_lift': [], 
                  'Total_margin_lift_percent_Recommended_Items': [], 'Total_margin_lift_percent_Category_level':[], 'Total_margin_lift_percent_Box_Level':[],
                 
                  'Pred_margin_rate': [], 'Recommended_margin_rate': [], 
                  'Margin_rate_lift': [], 'Margin_rate_lift_percent':[],
                  
                  'Current_avg_selling_price': [], 'Recommended_avg_selling_price': [],
                  'Min_price_change': [], 'Min_price_change_percent': [],
                  'Max_price_change': [], 'Max_price_change_percent': [],
                  'Avg_price_change': [], 'Avg_price_change_percent': [],
                  
                  'Current_price_gap_WMT': [], 'Recommended_price_gap_WMT': [],
                  'Current_price_gap_Costco': [], 'Recommended_price_gap_Costco': [],
                  'Current_price_gap_BJ': [], 'Recommended_price_gap_BJ': [],
                  }

In [0]:
#Finding the number of items recommended by each optimization model
items_count_MES = results_MES.select('scan_id')\
      .filter('Optimization_Flag_MES = 1').distinct().count()

RUM_summary_pd['Items_Recommended'].insert(0, items_count_MES)
print('MES # units:', items_count_MES)

items_count_MOO = results_MOO.select('scan_id')\
      .filter('Optimization_Flag_MOO = 1').distinct().count()

RUM_summary_pd['Items_Recommended'].insert(1, items_count_MOO)
print('MOO # units: ', items_count_MOO)

items_count_Units = results_Units.select('scan_id')\
      .filter('Optimization_Flag_Units = 1').distinct().count()

RUM_summary_pd['Items_Recommended'].insert(2, items_count_Units)
print('Units # units:', items_count_Units)

In [0]:
price_diff = results_MES\
      .select(['CBSA_MES', 'price_diff_MES', 'price_diff_percent_lift_MES', 
               'retail_MES', 'recommended_price_MES',
               'predicted_biannual_units_MES', 'predicted_units_MES'])\
      .filter('Optimization_Flag_MES = 1')\
      .where(F.col('price_diff_percent_lift_MES') <= 30)\
      .withColumn('current_sold_value', F.col('retail_MES')*F.col('predicted_biannual_units_MES'))\
      .withColumn('pred_sold_value', F.col('recommended_price_MES')*F.col('predicted_units_MES'))\

price_diff = price_diff\
      .groupBy('CBSA_MES')\
      .agg(F.max('price_diff_MES').alias('min_price_diff_MES'), 
           F.max('price_diff_percent_lift_MES').alias('min_price_lift_perc_MES'), 
           F.min('price_diff_MES').alias('max_price_diff_MES'), 
           F.min('price_diff_percent_lift_MES').alias('max_price_lift_perc_MES'), 
           F.mean('price_diff_MES').alias('avg_price_diff_MES'), 
           F.mean('price_diff_percent_lift_MES').alias('avg_price_lift_perc_MES'),
           (F.sum('current_sold_value')/F.sum('predicted_biannual_units_MES')).alias('avg_retail_MES'),
           (F.sum('pred_sold_value')/F.sum('predicted_units_MES')).alias('avg_recommended_price_MES')
          )

RUM_summary_pd['Current_avg_selling_price'].insert(0, price_diff.groupBy("CBSA_MES").mean().collect()[0]['avg(avg_retail_MES)'])
RUM_summary_pd['Recommended_avg_selling_price'].insert(0, price_diff.groupBy("CBSA_MES").mean().collect()[0]['avg(avg_recommended_price_MES)'])

RUM_summary_pd['Min_price_change'].insert(0, price_diff.groupBy("CBSA_MES").mean().collect()[0]['avg(min_price_diff_MES)'])
RUM_summary_pd['Min_price_change_percent'].insert(0, price_diff.groupBy("CBSA_MES").mean().collect()[0]['avg(min_price_lift_perc_MES)'])

RUM_summary_pd['Max_price_change'].insert(0, price_diff.groupBy("CBSA_MES").mean().collect()[0]['avg(max_price_diff_MES)'])
RUM_summary_pd['Max_price_change_percent'].insert(0, price_diff.groupBy("CBSA_MES").mean().collect()[0]['avg(max_price_lift_perc_MES)'])

RUM_summary_pd['Avg_price_change'].insert(0, price_diff.groupBy("CBSA_MES").mean().collect()[0]['avg(avg_price_diff_MES)'])
RUM_summary_pd['Avg_price_change_percent'].insert(0, price_diff.groupBy("CBSA_MES").mean().collect()[0]['avg(avg_price_lift_perc_MES)'])

display(price_diff)

In [0]:
price_diff = results_MOO\
      .select(['CBSA_MOO', 'price_diff_MOO', 'price_diff_percent_lift_MOO', 
               'retail_MOO', 'recommended_price_MOO',
               'predicted_biannual_units_MOO', 'predicted_units_MOO'])\
      .filter('Optimization_Flag_MOO = 1')\
      .where(F.col('price_diff_percent_lift_MOO') <= 30)\
      .withColumn('current_sold_value', F.col('retail_MOO')*F.col('predicted_biannual_units_MOO'))\
      .withColumn('pred_sold_value', F.col('recommended_price_MOO')*F.col('predicted_units_MOO'))\

price_diff = price_diff\
      .groupBy('CBSA_MOO')\
      .agg(F.max('price_diff_MOO').alias('min_price_diff_MOO'), 
           F.max('price_diff_percent_lift_MOO').alias('min_price_lift_perc_MOO'), 
           F.min('price_diff_MOO').alias('max_price_diff_MOO'), 
           F.min('price_diff_percent_lift_MOO').alias('max_price_lift_perc_MOO'), 
           F.mean('price_diff_MOO').alias('avg_price_diff_MOO'), 
           F.mean('price_diff_percent_lift_MOO').alias('avg_price_lift_perc_MOO'),
           (F.sum('current_sold_value')/F.sum('predicted_biannual_units_MOO')).alias('avg_retail_MOO'),
           (F.sum('pred_sold_value')/F.sum('predicted_units_MOO')).alias('avg_recommended_price_MOO')
          )

RUM_summary_pd['Current_avg_selling_price'].insert(1, price_diff.groupBy("CBSA_MOO").mean().collect()[0]['avg(avg_retail_MOO)'])
RUM_summary_pd['Recommended_avg_selling_price'].insert(1, price_diff.groupBy("CBSA_MOO").mean().collect()[0]['avg(avg_recommended_price_MOO)'])

RUM_summary_pd['Min_price_change'].insert(1, price_diff.groupBy("CBSA_MOO").mean().collect()[0]['avg(min_price_diff_MOO)'])
RUM_summary_pd['Min_price_change_percent'].insert(1, price_diff.groupBy("CBSA_MOO").mean().collect()[0]['avg(min_price_lift_perc_MOO)'])

RUM_summary_pd['Max_price_change'].insert(1, price_diff.groupBy("CBSA_MOO").mean().collect()[0]['avg(max_price_diff_MOO)'])
RUM_summary_pd['Max_price_change_percent'].insert(1, price_diff.groupBy("CBSA_MOO").mean().collect()[0]['avg(max_price_lift_perc_MOO)'])

RUM_summary_pd['Avg_price_change'].insert(1, price_diff.groupBy("CBSA_MOO").mean().collect()[0]['avg(avg_price_diff_MOO)'])
RUM_summary_pd['Avg_price_change_percent'].insert(1, price_diff.groupBy("CBSA_MOO").mean().collect()[0]['avg(avg_price_lift_perc_MOO)'])
# display(price_diff)

In [0]:
price_diff = results_Units\
      .select(['CBSA_Units', 'price_diff_Units', 'price_diff_percent_lift_Units', 
               'retail_Units', 'recommended_price_Units',
               'predicted_biannual_units_Units', 'predicted_units_Units'])\
      .filter('Optimization_Flag_Units = 1')\
      .where(F.col('price_diff_percent_lift_Units') <= 30)\
      .withColumn('current_sold_value', F.col('retail_Units')*F.col('predicted_biannual_units_Units'))\
      .withColumn('pred_sold_value', F.col('recommended_price_Units')*F.col('predicted_units_Units'))\

price_diff = price_diff\
      .groupBy('CBSA_Units')\
      .agg(F.max('price_diff_Units').alias('min_price_diff_Units'), 
           F.max('price_diff_percent_lift_Units').alias('min_price_lift_perc_Units'), 
           F.min('price_diff_Units').alias('max_price_diff_Units'), 
           F.min('price_diff_percent_lift_Units').alias('max_price_lift_perc_Units'), 
           F.mean('price_diff_Units').alias('avg_price_diff_Units'), 
           F.mean('price_diff_percent_lift_Units').alias('avg_price_lift_perc_Units'),
           (F.sum('current_sold_value')/F.sum('predicted_biannual_units_Units')).alias('avg_retail_Units'),
           (F.sum('pred_sold_value')/F.sum('predicted_units_Units')).alias('avg_recommended_price_Units')
          )

RUM_summary_pd['Current_avg_selling_price'].insert(2, price_diff.groupBy("CBSA_Units").mean().collect()[0]['avg(avg_retail_Units)'])
RUM_summary_pd['Recommended_avg_selling_price'].insert(2, price_diff.groupBy("CBSA_Units").mean().collect()[0]['avg(avg_recommended_price_Units)'])

RUM_summary_pd['Min_price_change'].insert(2, price_diff.groupBy("CBSA_Units").mean().collect()[0]['avg(min_price_diff_Units)'])
RUM_summary_pd['Min_price_change_percent'].insert(2, price_diff.groupBy("CBSA_Units").mean().collect()[0]['avg(min_price_lift_perc_Units)'])

RUM_summary_pd['Max_price_change'].insert(2, price_diff.groupBy("CBSA_Units").mean().collect()[0]['avg(max_price_diff_Units)'])
RUM_summary_pd['Max_price_change_percent'].insert(2, price_diff.groupBy("CBSA_Units").mean().collect()[0]['avg(max_price_lift_perc_Units)'])

RUM_summary_pd['Avg_price_change'].insert(2, price_diff.groupBy("CBSA_Units").mean().collect()[0]['avg(avg_price_diff_Units)'])
RUM_summary_pd['Avg_price_change_percent'].insert(2, price_diff.groupBy("CBSA_Units").mean().collect()[0]['avg(avg_price_lift_perc_Units)'])
# display(price_diff)

In [0]:
# Units lift for MES optimization
actual_units_MES_init = results_MES.select('predicted_biannual_units_MES').toPandas()
actual_units_MES = sum(actual_units_MES_init['predicted_biannual_units_MES'].values)

price_points_units_MES_init = results_MES.select('predicted_units_MES').toPandas()
price_points_units_MES = sum(price_points_units_MES_init['predicted_units_MES'].values)

df_units_MES = results_MES.select('units_diff_MES').toPandas()
units_lift_MES = sum(df_units_MES['units_diff_MES'].values)

RUM_summary_pd['Current_units_volume'].insert(0, actual_units_MES)
RUM_summary_pd['Recommended_units_volume'].insert(0, price_points_units_MES)
RUM_summary_pd['Units_lift'].insert(0, units_lift_MES)

print("MES optimization gives {} units lift".format(units_lift_MES))

# Units lift for MOO optimization
actual_units_MOO_init = results_MOO.select('predicted_biannual_units_MOO').toPandas()
actual_units_MOO = sum(actual_units_MOO_init['predicted_biannual_units_MOO'].values)

price_points_units_MOO_init = results_MOO.select('predicted_units_MOO').toPandas()
price_points_units_MOO = sum(price_points_units_MOO_init['predicted_units_MOO'].values)

df_units_MOO = results_MOO.select('units_diff_MOO').toPandas()
units_lift_MOO = sum(df_units_MOO['units_diff_MOO'].values)

RUM_summary_pd['Current_units_volume'].insert(1, actual_units_MOO)
RUM_summary_pd['Recommended_units_volume'].insert(1, price_points_units_MOO)
RUM_summary_pd['Units_lift'].insert(1, units_lift_MOO)

print("MOO optimization gives {} units lift".format(units_lift_MOO))

# Units lift for Units optimization
actual_units_Units_init = results_Units.select('predicted_biannual_units_Units').toPandas()
actual_units_Units = sum(actual_units_Units_init['predicted_biannual_units_Units'].values)

price_points_units_Units_init = results_Units.select('predicted_units_Units').toPandas()
price_points_units_Units = sum(price_points_units_Units_init['predicted_units_Units'].values)

df_units_Units = results_Units.select('units_diff_Units').toPandas()
units_lift_Units = sum(df_units_Units['units_diff_Units'].values)

RUM_summary_pd['Current_units_volume'].insert(2, actual_units_Units)
RUM_summary_pd['Recommended_units_volume'].insert(2, price_points_units_Units)
RUM_summary_pd['Units_lift'].insert(2, units_lift_Units)

print("Units optimization gives {} units lift".format(units_lift_Units))

In [0]:
# Use the data from the all items table to calculate the RUM analysis
# with respect to category level, and box level, utilizing the Item_Status_Flag
# that helps categorgirzing that data in different levels:
# optimized items, the remaining items in the same categories, and all other items in the clubs
box_all_items_prediction_df = spark.read.table(all_items_file)
# box_all_items_prediction_df = df_all_items_details

df_all_items_details = df_all_items_details\
      .filter((F.col("Item_status_flag") == 'pi_item_optimization_feasible') | (F.col("Item_status_flag") == 'pi_item_optimization_infeasible') | (F.col("Item_status_flag") == 'non_pi_item_target_categor') | (F.col("Item_status_flag") == 'Optimization_picked'))

# level 2 items: categories level
total_units_cat = (df_all_items_details\
                   .select(['Total_Units_Sold'])\
                   .groupBy().sum()).toPandas().iloc[0,0]


# level 3 items: box level
total_units = (box_all_items_prediction_df[['Total_Units_Sold']]\
               .groupBy().sum()).toPandas().iloc[0,0]

# MES units lift %
units_lift_perc_recommended_items_MES = units_lift_MES/results_MES[['predicted_biannual_units_MES']].groupBy().sum().toPandas().iloc[0,0]
units_lift_cat_perc_MES = units_lift_MES/total_units_cat
units_lift_perc_MES = units_lift_MES/total_units

RUM_summary_pd['Units_lift_Percent_Recommended_Items'].insert(0, units_lift_perc_recommended_items_MES)
RUM_summary_pd['Units_lift_Percent_Category_Level'].insert(0, units_lift_cat_perc_MES)
RUM_summary_pd['Units_lift_Percent_Box_Level'].insert(0, units_lift_perc_MES)

print('Units lift for recommended items MES opt is: {}'.format(units_lift_perc_recommended_items_MES))
print('Units lift for all cats MES Opt is: {}'.format(units_lift_cat_perc_MES))
print('Units lift for all items MES Opt is: {}'.format(units_lift_perc_MES))

# MOO units lift %
units_lift_perc_recommended_items_MOO = units_lift_MOO/ results_MOO[['predicted_biannual_units_MOO']].groupBy().sum().toPandas().iloc[0,0]
units_lift_cat_perc_MOO = units_lift_MOO/total_units_cat
units_lift_perc_MOO = units_lift_MOO/total_units

RUM_summary_pd['Units_lift_Percent_Recommended_Items'].insert(1, units_lift_perc_recommended_items_MOO)
RUM_summary_pd['Units_lift_Percent_Category_Level'].insert(1, units_lift_cat_perc_MOO)
RUM_summary_pd['Units_lift_Percent_Box_Level'].insert(1, units_lift_perc_MOO)

print('')
print('Units lift for recommended items MOO opt is: {}'.format(units_lift_perc_recommended_items_MOO))
print('Units lift for MOO Opt is: {}'.format(units_lift_perc_MOO))
print('Units lift for all cats MOO Opt is: {}'.format(units_lift_cat_perc_MOO))

# Units units lift %
units_lift_perc_recommended_items_Units = units_lift_Units/results_Units[['predicted_biannual_units_Units']].groupBy().sum().toPandas().iloc[0,0]
units_lift_cat_perc_Units = units_lift_Units/total_units_cat
units_lift_perc_Units = units_lift_Units/total_units

RUM_summary_pd['Units_lift_Percent_Recommended_Items'].insert(2, units_lift_perc_recommended_items_Units)
RUM_summary_pd['Units_lift_Percent_Category_Level'].insert(2, units_lift_cat_perc_Units)
RUM_summary_pd['Units_lift_Percent_Box_Level'].insert(2, units_lift_perc_Units)

print('')
print('Units lift for recommended items Units opt is: {}'.format(units_lift_perc_recommended_items_Units))
print('Units lift for Units Opt is: {}'.format(units_lift_perc_Units))
print('Units lift for all cats Units Opt is: {}'.format(units_lift_cat_perc_Units))

In [0]:
print(total_units_cat)
print(total_units)

In [0]:
print(units_lift_MES)
print(results_MES[['predicted_biannual_units_MES']].groupBy().sum().toPandas().iloc[0,0])
print(results_MES.select('predicted_biannual_units_MES').count())

In [0]:
# code block to calculate profit and revenue for all items 
# (from the forecast table of all items in specified clubs)
# profit = selling_price * unit_qty -  unit_cost * unit_qty
# revenue = selling_price * unit_qty

total_profit_cat = (df_all_items_details[['actual_biannual_profit']].groupBy().sum()).toPandas().iloc[0,0]
total_profit_all_items = (box_all_items_prediction_df[['actual_biannual_profit']].groupBy().sum()).toPandas().iloc[0,0]

total_revenue_cat = (df_all_items_details[['actual_biannual_revenue']].groupBy().sum()).toPandas().iloc[0,0]
total_revenue_all_items = (box_all_items_prediction_df[['actual_biannual_revenue']].groupBy().sum()).toPandas().iloc[0,0]

In [0]:
# This is for all items
print('profit categories: ', total_profit_cat)
print('revenue categories: ', total_revenue_cat)

print('profit all items: ', total_profit_all_items)
print('revenue all items: ', total_revenue_all_items)

In [0]:
df_units_MES = results_MES.select(['Optimization_Flag_MES', 'profit_diff_MES', 'revenue_diff_MES', 
                                   'predicted_profit_MES', 'predicted_revenue_MES',
                                   'predicted_biannual_profit_MES', 'predicted_biannual_revenue_MES']).toPandas()

profit_lift_recommended_items_MES = sum(df_units_MES['profit_diff_MES'].values)/sum(df_units_MES['predicted_biannual_profit_MES'].values)
revenue_lift_recommended_items_MES = sum(df_units_MES['revenue_diff_MES'].values)/sum(df_units_MES['predicted_biannual_revenue_MES'].values)

profit_lift_MES = sum(df_units_MES['profit_diff_MES'].values)/total_profit_cat
revenue_lift_MES = sum(df_units_MES['revenue_diff_MES'].values)/total_revenue_cat

profit_lift_MES_cat_all = sum(df_units_MES['profit_diff_MES'].values)/total_profit_all_items
revenue_lift_MES_cat_all = sum(df_units_MES['revenue_diff_MES'].values)/total_revenue_all_items

RUM_summary_pd['Total_margin_lift_percent_Recommended_Items'].insert(0, profit_lift_recommended_items_MES)
RUM_summary_pd['Total_sales_lift_percent_Recommended_Items'].insert(0, revenue_lift_recommended_items_MES)

RUM_summary_pd['Total_margin_lift_percent_Category_level'].insert(0, profit_lift_MES)
RUM_summary_pd['Total_sales_lift_percent_Category_Level'].insert(0, revenue_lift_MES)

RUM_summary_pd['Total_margin_lift_percent_Box_Level'].insert(0, profit_lift_MES_cat_all)
RUM_summary_pd['Total_sales_lift_percent_Box_Level'].insert(0, revenue_lift_MES_cat_all)

print('MES profit increases recommended items: ', profit_lift_recommended_items_MES)
print('MES revenue increases recommended items: ', revenue_lift_recommended_items_MES)
print("MES profit increased by: ", profit_lift_MES)
print('MES revenue increased by: ', revenue_lift_MES)
print("MES profit, all cat, increased by: ", profit_lift_MES_cat_all)
print('MES revenue, all cat, increased by: ', revenue_lift_MES_cat_all)


df_units_MOO = results_MOO.select(['Optimization_Flag_MOO', 'profit_diff_MOO', 'revenue_diff_MOO','predicted_biannual_profit_MOO', 
                                   'predicted_profit_MOO', 'predicted_revenue_MOO',
                                   'predicted_biannual_revenue_MOO']).toPandas()
profit_lift_recommended_items_MOO = sum(df_units_MOO['profit_diff_MOO'].values)/sum(df_units_MOO['predicted_biannual_profit_MOO'].values)
revenue_lift_recommended_items_MOO = sum(df_units_MOO['revenue_diff_MOO'].values)/sum(df_units_MOO['predicted_biannual_revenue_MOO'].values)

profit_lift_MOO = sum(df_units_MOO['profit_diff_MOO'].values)/total_profit_cat
revenue_lift_MOO = sum(df_units_MOO['revenue_diff_MOO'].values)/total_revenue_cat

profit_lift_MOO_cat_all = sum(df_units_MOO['profit_diff_MOO'].values)/total_profit_all_items
revenue_lift_MOO_cat_all = sum(df_units_MOO['revenue_diff_MOO'].values)/total_revenue_all_items

RUM_summary_pd['Total_sales_lift_percent_Recommended_Items'].insert(1, revenue_lift_recommended_items_MOO)
RUM_summary_pd['Total_margin_lift_percent_Recommended_Items'].insert(1, profit_lift_recommended_items_MOO)

RUM_summary_pd['Total_margin_lift_percent_Category_level'].insert(1, profit_lift_MOO)
RUM_summary_pd['Total_sales_lift_percent_Category_Level'].insert(1, revenue_lift_MOO)

RUM_summary_pd['Total_margin_lift_percent_Box_Level'].insert(1, profit_lift_MOO_cat_all)
RUM_summary_pd['Total_sales_lift_percent_Box_Level'].insert(1, revenue_lift_MOO_cat_all)

print('')
print('MOO profit increases recommended items: ', profit_lift_recommended_items_MOO)
print('MOO revenue increases recommended items: ', revenue_lift_recommended_items_MOO)
print("MOO profit increased by: ", profit_lift_MOO)
print('MOO revenue increased by: ', revenue_lift_MOO)
print("MOO profit, all cat, increased by: ", profit_lift_MOO_cat_all)
print('MOO revenue, all cat, increased by: ', revenue_lift_MOO_cat_all)

df_units_Units = results_Units.select(['Optimization_Flag_Units', 'profit_diff_Units', 'revenue_diff_Units', 
                                       'predicted_profit_Units', 'predicted_revenue_Units',
                                       'predicted_biannual_profit_Units', 'predicted_biannual_revenue_Units']).toPandas()
profit_lift_recommended_items_Units = sum(df_units_Units['profit_diff_Units'].values)/sum(df_units_Units['predicted_biannual_profit_Units'].values)
revenue_lift_recommended_items_Units = sum(df_units_Units['revenue_diff_Units'].values)/sum(df_units_Units['predicted_biannual_revenue_Units'].values)

profit_lift_Units = sum(df_units_Units['profit_diff_Units'].values)/total_profit_cat
revenue_lift_Units = sum(df_units_Units['revenue_diff_Units'].values)/total_revenue_cat

profit_lift_Units_cat_all = sum(df_units_Units['profit_diff_Units'].values)/total_profit_all_items
revenue_lift_Units_cat_all = sum(df_units_Units['revenue_diff_Units'].values)/total_revenue_all_items

RUM_summary_pd['Total_sales_lift_percent_Recommended_Items'].insert(2, revenue_lift_recommended_items_Units)
RUM_summary_pd['Total_margin_lift_percent_Recommended_Items'].insert(2, profit_lift_recommended_items_Units)

RUM_summary_pd['Total_margin_lift_percent_Category_level'].insert(2, profit_lift_Units)
RUM_summary_pd['Total_sales_lift_percent_Category_Level'].insert(2, revenue_lift_Units)

RUM_summary_pd['Total_margin_lift_percent_Box_Level'].insert(2, profit_lift_Units_cat_all)
RUM_summary_pd['Total_sales_lift_percent_Box_Level'].insert(2, revenue_lift_Units_cat_all)

print('')
print('Units profit increases recommended items: ', profit_lift_recommended_items_Units)
print('Units revenue increases recommended items: ', revenue_lift_recommended_items_Units)
print("Units profit increased by: ", profit_lift_Units)
print('Units revenue increased by: ', revenue_lift_Units)
print("Units profit, all cat, increased by: ", profit_lift_Units_cat_all)
print('Units revenue, all cat, increased by: ', revenue_lift_Units_cat_all)


In [0]:
# df = spark.read.table('s0a02qd.RUM_analysis_MES_MOO_Units_StL_30MBudget_1YearElas_MES_NN')
# df_new = df.na.fill(-9000000000)
# df_new = df_new.na.fill("NULL")
# display(df_new)

In [0]:
pred_total_revenue_MES = sum(df_units_MES['predicted_biannual_revenue_MES'].values)
pred_total_revenue_price_points_MES = sum(df_units_MES['predicted_revenue_MES'].values)
revenue_lift_MES = sum(df_units_MES['revenue_diff_MES'].values)

pred_total_profit_MES = sum(df_units_MES['predicted_biannual_profit_MES'].values)
pred_total_profit_price_points_MES = sum(df_units_MES['predicted_profit_MES'].values)
profit_lift_MES = sum(df_units_MES['profit_diff_MES'].values)

RUM_summary_pd['Pred_total_sales'].insert(0, pred_total_revenue_MES)
RUM_summary_pd['Recommended_total_sales'].insert(0, pred_total_revenue_price_points_MES)
RUM_summary_pd['Total_sales_lift'].insert(0, revenue_lift_MES)

RUM_summary_pd['Pred_total_margin'].insert(0, pred_total_profit_MES)
RUM_summary_pd['Recommended_total_margin'].insert(0, pred_total_profit_price_points_MES)
RUM_summary_pd['Total_margin_lift'].insert(0, profit_lift_MES)

print('MES profit $ value: ', profit_lift_MES)
print('MES revenue $ value: ', revenue_lift_MES)

#============================================================================================================
pred_total_revenue_MOO = sum(df_units_MOO['predicted_biannual_revenue_MOO'].values)
pred_total_revenue_price_points_MOO = sum(df_units_MOO['predicted_revenue_MOO'].values)
revenue_lift_MOO = sum(df_units_MOO['revenue_diff_MOO'].values)

pred_total_profit_MOO = sum(df_units_MOO['predicted_biannual_profit_MOO'].values)
pred_total_profit_price_points_MOO = sum(df_units_MOO['predicted_profit_MOO'].values)
profit_lift_MOO = sum(df_units_MOO['profit_diff_MOO'].values)

RUM_summary_pd['Pred_total_sales'].insert(1, pred_total_revenue_MOO)
RUM_summary_pd['Recommended_total_sales'].insert(1, pred_total_revenue_price_points_MOO)
RUM_summary_pd['Total_sales_lift'].insert(1, revenue_lift_MOO)

RUM_summary_pd['Pred_total_margin'].insert(1, pred_total_profit_MOO)
RUM_summary_pd['Recommended_total_margin'].insert(1, pred_total_profit_price_points_MOO)
RUM_summary_pd['Total_margin_lift'].insert(1, profit_lift_MOO)

print('MOO profit $ value: ', profit_lift_MOO)
print('MOO revenue $ value: ', revenue_lift_MOO)

#============================================================================================================
pred_total_revenue_Units = sum(df_units_Units['predicted_biannual_revenue_Units'].values)
pred_total_revenue_price_points_Units = sum(df_units_Units['predicted_revenue_Units'].values)
revenue_lift_Units = sum(df_units_Units['revenue_diff_Units'].values)

pred_total_profit_Units = sum(df_units_Units['predicted_biannual_profit_Units'].values)
pred_total_profit_price_points_Units = sum(df_units_Units['predicted_profit_Units'].values)
profit_lift_Units = sum(df_units_Units['profit_diff_Units'].values)

RUM_summary_pd['Pred_total_sales'].insert(2, pred_total_revenue_Units)
RUM_summary_pd['Recommended_total_sales'].insert(2, pred_total_revenue_price_points_Units)
RUM_summary_pd['Total_sales_lift'].insert(2, revenue_lift_Units)

RUM_summary_pd['Pred_total_margin'].insert(2, pred_total_profit_Units)
RUM_summary_pd['Recommended_total_margin'].insert(2, pred_total_profit_price_points_Units)
RUM_summary_pd['Total_margin_lift'].insert(2, profit_lift_Units)

print('Units profit $ value: ', profit_lift_Units)
print('Units revenue $ value: ', revenue_lift_Units)

In [0]:
pred_total_revenue_MES = sum(df_units_MES[df_units_MES['Optimization_Flag_MES'] == 1]['predicted_biannual_revenue_MES'].values)
pred_total_revenue_price_points_MES = sum(df_units_MES[df_units_MES['Optimization_Flag_MES'] == 1]['predicted_revenue_MES'].values)
revenue_lift_MES = sum(df_units_MES[df_units_MES['Optimization_Flag_MES'] == 1]['revenue_diff_MES'].values)

pred_total_profit_MES = sum(df_units_MES[df_units_MES['Optimization_Flag_MES'] == 1]['predicted_biannual_profit_MES'].values)
pred_total_profit_price_points_MES = sum(df_units_MES[df_units_MES['Optimization_Flag_MES'] == 1]['predicted_profit_MES'].values)
profit_lift_MES = sum(df_units_MES[df_units_MES['Optimization_Flag_MES'] == 1]['profit_diff_MES'].values)

pred_margin_rate_MES = pred_total_profit_MES/pred_total_revenue_MES
pred_margin_rate_price_points_MES = pred_total_profit_price_points_MES/pred_total_revenue_price_points_MES
margin_rate_lift_MES = pred_margin_rate_price_points_MES - pred_margin_rate_MES
margin_rate_lift_percent_MES = margin_rate_lift_MES/pred_margin_rate_MES

RUM_summary_pd['Pred_margin_rate'].insert(0, pred_margin_rate_MES)
RUM_summary_pd['Recommended_margin_rate'].insert(0, pred_margin_rate_price_points_MES)
RUM_summary_pd['Margin_rate_lift'].insert(0, margin_rate_lift_MES)
RUM_summary_pd['Margin_rate_lift_percent'].insert(0, margin_rate_lift_percent_MES)

print('MES profit $ value: ', profit_lift_MES)
print('MES revenue $ value: ', revenue_lift_MES)

#============================================================================================================
pred_total_revenue_MOO = sum(df_units_MOO[df_units_MOO['Optimization_Flag_MOO'] == 1]['predicted_biannual_revenue_MOO'].values)
pred_total_revenue_price_points_MOO = sum(df_units_MOO[df_units_MOO['Optimization_Flag_MOO'] == 1]['predicted_revenue_MOO'].values)
revenue_lift_MOO = sum(df_units_MOO[df_units_MOO['Optimization_Flag_MOO'] == 1]['revenue_diff_MOO'].values)

pred_total_profit_MOO = sum(df_units_MOO[df_units_MOO['Optimization_Flag_MOO'] == 1]['predicted_biannual_profit_MOO'].values)
pred_total_profit_price_points_MOO = sum(df_units_MOO[df_units_MOO['Optimization_Flag_MOO'] == 1]['predicted_profit_MOO'].values)
profit_lift_MOO = sum(df_units_MOO[df_units_MOO['Optimization_Flag_MOO'] == 1]['profit_diff_MOO'].values)

pred_margin_rate_MOO = pred_total_profit_MOO/pred_total_revenue_MOO
pred_margin_rate_price_points_MOO = pred_total_profit_price_points_MOO/pred_total_revenue_price_points_MOO
margin_rate_lift_MOO = pred_margin_rate_price_points_MOO - pred_margin_rate_MOO
margin_rate_lift_percent_MOO = margin_rate_lift_MOO/pred_margin_rate_MOO

RUM_summary_pd['Pred_margin_rate'].insert(1, pred_margin_rate_MOO)
RUM_summary_pd['Recommended_margin_rate'].insert(1, pred_margin_rate_price_points_MOO)
RUM_summary_pd['Margin_rate_lift'].insert(1, margin_rate_lift_MOO)
RUM_summary_pd['Margin_rate_lift_percent'].insert(1, margin_rate_lift_percent_MOO)

print('MOO profit $ value: ', profit_lift_MOO)
print('MOO revenue $ value: ', revenue_lift_MOO)

#============================================================================================================
pred_total_revenue_Units = sum(df_units_Units[df_units_Units['Optimization_Flag_Units'] == 1]['predicted_biannual_revenue_Units'].values)
pred_total_revenue_price_points_Units = sum(df_units_Units[df_units_Units['Optimization_Flag_Units'] == 1]['predicted_revenue_Units'].values)
revenue_lift_Units = sum(df_units_Units[df_units_Units['Optimization_Flag_Units'] == 1]['revenue_diff_Units'].values)

pred_total_profit_Units = sum(df_units_Units[df_units_Units['Optimization_Flag_Units'] == 1]['predicted_biannual_profit_Units'].values)
pred_total_profit_price_points_Units = sum(df_units_Units[df_units_Units['Optimization_Flag_Units'] == 1]['predicted_profit_Units'].values)
profit_lift_Units = sum(df_units_Units[df_units_Units['Optimization_Flag_Units'] == 1]['profit_diff_Units'].values)

pred_margin_rate_Units = pred_total_profit_Units/pred_total_revenue_Units
pred_margin_rate_price_points_Units = pred_total_profit_price_points_Units/pred_total_revenue_price_points_Units
margin_rate_lift_Units = pred_margin_rate_price_points_Units - pred_margin_rate_Units
margin_rate_lift_percent_Units = margin_rate_lift_Units/pred_margin_rate_Units

RUM_summary_pd['Pred_margin_rate'].insert(2, pred_margin_rate_Units)
RUM_summary_pd['Recommended_margin_rate'].insert(2, pred_margin_rate_price_points_Units)
RUM_summary_pd['Margin_rate_lift'].insert(2, margin_rate_lift_Units)
RUM_summary_pd['Margin_rate_lift_percent'].insert(2, margin_rate_lift_percent_Units)

print('Units profit $ value: ', profit_lift_Units)
print('Units revenue $ value: ', revenue_lift_Units)

In [0]:
price_gaps_MES = results_MES.select(['Optimization_Flag_MES', 'CBSA_MES', 'predicted_biannual_units_MES', 'predicted_units_MES',
                                          'WMT_Price_Gap_before_MES', 'COS_Price_Gap_before_MES', 'BJ_Price_Gap_before_MES',
                                          'WMT_Price_Gap_after_MES', 'COS_Price_Gap_after_MES', 'BJ_Price_Gap_after_MES'])\
      .filter('Optimization_Flag_MES = 1')\
      .withColumn('wmt_price_gap_before', F.col('WMT_Price_Gap_before_MES') * F.col('predicted_biannual_units_MES'))\
      .withColumn('wmt_price_gap_after', F.col('WMT_Price_Gap_after_MES') * F.col('predicted_units_MES'))\
      .withColumn('cos_price_gap_before', F.col('COS_Price_Gap_before_MES') * F.col('predicted_biannual_units_MES'))\
      .withColumn('cos_price_gap_after', F.col('COS_Price_Gap_after_MES') * F.col('predicted_units_MES'))\
      .withColumn('bj_price_gap_before', F.col('BJ_Price_Gap_before_MES') * F.col('predicted_biannual_units_MES'))\
      .withColumn('bj_price_gap_after', F.col('BJ_Price_Gap_after_MES') * F.col('predicted_units_MES'))\


price_gaps_MES = price_gaps_MES\
      .groupBy('CBSA_MES')\
      .agg((F.sum('wmt_price_gap_before')/F.sum('predicted_biannual_units_MES')).alias('weighted_avg_WMT_Price_Gap_before_MES'),
           (F.sum('wmt_price_gap_after')/F.sum('predicted_units_MES')).alias('weighted_avg_WMT_Price_Gap_after_MES'),
           (F.sum('cos_price_gap_before')/F.sum('predicted_biannual_units_MES')).alias('weighted_avg_COS_Price_Gap_before_MES'),
           (F.sum('cos_price_gap_after')/F.sum('predicted_units_MES')).alias('weighted_avg_COS_Price_Gap_after_MES'),
           (F.sum('bj_price_gap_before')/F.sum('predicted_biannual_units_MES')).alias('weighted_avg_BJ_Price_Gap_before_MES'),
           (F.sum('bj_price_gap_after')/F.sum('predicted_units_MES')).alias('weighted_avg_BJ_Price_Gap_after_MES')
          )
           
RUM_summary_pd['Current_price_gap_WMT'].insert(0, price_gaps_MES.groupBy("CBSA_MES").mean().collect()[0]['avg(weighted_avg_WMT_Price_Gap_before_MES)'])
RUM_summary_pd['Recommended_price_gap_WMT'].insert(0, price_gaps_MES.groupBy("CBSA_MES").mean().collect()[0]['avg(weighted_avg_WMT_Price_Gap_after_MES)'])

RUM_summary_pd['Current_price_gap_Costco'].insert(0, price_gaps_MES.groupBy("CBSA_MES").mean().collect()[0]['avg(weighted_avg_COS_Price_Gap_before_MES)'])
RUM_summary_pd['Recommended_price_gap_Costco'].insert(0, price_gaps_MES.groupBy("CBSA_MES").mean().collect()[0]['avg(weighted_avg_COS_Price_Gap_after_MES)'])

RUM_summary_pd['Current_price_gap_BJ'].insert(0, price_gaps_MES.groupBy("CBSA_MES").mean().collect()[0]['avg(weighted_avg_BJ_Price_Gap_before_MES)'])
RUM_summary_pd['Recommended_price_gap_BJ'].insert(0, price_gaps_MES.groupBy("CBSA_MES").mean().collect()[0]['avg(weighted_avg_BJ_Price_Gap_after_MES)'])

#===========================================
price_gaps_MOO = results_MOO.select(['Optimization_Flag_MOO', 'CBSA_MOO', 'predicted_biannual_units_MOO', 'predicted_units_MOO',
                                          'WMT_Price_Gap_before_MOO', 'COS_Price_Gap_before_MOO', 'BJ_Price_Gap_before_MOO',
                                          'WMT_Price_Gap_after_MOO', 'COS_Price_Gap_after_MOO', 'BJ_Price_Gap_after_MOO'])\
      .filter('Optimization_Flag_MOO = 1')\
      .withColumn('wmt_price_gap_before', F.col('WMT_Price_Gap_before_MOO') * F.col('predicted_biannual_units_MOO'))\
      .withColumn('wmt_price_gap_after', F.col('WMT_Price_Gap_after_MOO') * F.col('predicted_units_MOO'))\
      .withColumn('cos_price_gap_before', F.col('COS_Price_Gap_before_MOO') * F.col('predicted_biannual_units_MOO'))\
      .withColumn('cos_price_gap_after', F.col('COS_Price_Gap_after_MOO') * F.col('predicted_units_MOO'))\
      .withColumn('bj_price_gap_before', F.col('BJ_Price_Gap_before_MOO') * F.col('predicted_biannual_units_MOO'))\
      .withColumn('bj_price_gap_after', F.col('BJ_Price_Gap_after_MOO') * F.col('predicted_units_MOO'))\


price_gaps_MOO = price_gaps_MOO\
      .groupBy('CBSA_MOO')\
      .agg((F.sum('wmt_price_gap_before')/F.sum('predicted_biannual_units_MOO')).alias('weighted_avg_WMT_Price_Gap_before_MOO'),
           (F.sum('wmt_price_gap_after')/F.sum('predicted_units_MOO')).alias('weighted_avg_WMT_Price_Gap_after_MOO'),
           (F.sum('cos_price_gap_before')/F.sum('predicted_biannual_units_MOO')).alias('weighted_avg_COS_Price_Gap_before_MOO'),
           (F.sum('cos_price_gap_after')/F.sum('predicted_units_MOO')).alias('weighted_avg_COS_Price_Gap_after_MOO'),
           (F.sum('bj_price_gap_before')/F.sum('predicted_biannual_units_MOO')).alias('weighted_avg_BJ_Price_Gap_before_MOO'),
           (F.sum('bj_price_gap_after')/F.sum('predicted_units_MOO')).alias('weighted_avg_BJ_Price_Gap_after_MOO'))
           
RUM_summary_pd['Current_price_gap_WMT'].insert(1, price_gaps_MOO.groupBy("CBSA_MOO").mean().collect()[0]['avg(weighted_avg_WMT_Price_Gap_before_MOO)'])
RUM_summary_pd['Recommended_price_gap_WMT'].insert(1, price_gaps_MOO.groupBy("CBSA_MOO").mean().collect()[0]['avg(weighted_avg_WMT_Price_Gap_after_MOO)'])

RUM_summary_pd['Current_price_gap_Costco'].insert(1, price_gaps_MOO.groupBy("CBSA_MOO").mean().collect()[0]['avg(weighted_avg_COS_Price_Gap_before_MOO)'])
RUM_summary_pd['Recommended_price_gap_Costco'].insert(1, price_gaps_MOO.groupBy("CBSA_MOO").mean().collect()[0]['avg(weighted_avg_COS_Price_Gap_after_MOO)'])

RUM_summary_pd['Current_price_gap_BJ'].insert(1, price_gaps_MOO.groupBy("CBSA_MOO").mean().collect()[0]['avg(weighted_avg_BJ_Price_Gap_before_MOO)'])
RUM_summary_pd['Recommended_price_gap_BJ'].insert(1, price_gaps_MOO.groupBy("CBSA_MOO").mean().collect()[0]['avg(weighted_avg_BJ_Price_Gap_after_MOO)'])

#===========================================
price_gaps_Units = results_Units.select(['Optimization_Flag_Units', 'CBSA_Units', 'predicted_biannual_units_Units', 'predicted_units_Units',
                                          'WMT_Price_Gap_before_Units', 'COS_Price_Gap_before_Units', 'BJ_Price_Gap_before_Units',
                                          'WMT_Price_Gap_after_Units', 'COS_Price_Gap_after_Units', 'BJ_Price_Gap_after_Units'])\
      .filter('Optimization_Flag_Units = 1')\
      .withColumn('wmt_price_gap_before', F.col('WMT_Price_Gap_before_Units') * F.col('predicted_biannual_units_Units'))\
      .withColumn('wmt_price_gap_after', F.col('WMT_Price_Gap_after_Units') * F.col('predicted_units_Units'))\
      .withColumn('cos_price_gap_before', F.col('COS_Price_Gap_before_Units') * F.col('predicted_biannual_units_Units'))\
      .withColumn('cos_price_gap_after', F.col('COS_Price_Gap_after_Units') * F.col('predicted_units_Units'))\
      .withColumn('bj_price_gap_before', F.col('BJ_Price_Gap_before_Units') * F.col('predicted_biannual_units_Units'))\
      .withColumn('bj_price_gap_after', F.col('BJ_Price_Gap_after_Units') * F.col('predicted_units_Units'))\


price_gaps_Units = price_gaps_Units\
      .groupBy('CBSA_Units')\
      .agg((F.sum('wmt_price_gap_before')/F.sum('predicted_biannual_units_Units')).alias('weighted_avg_WMT_Price_Gap_before_Units'),
           (F.sum('wmt_price_gap_after')/F.sum('predicted_units_Units')).alias('weighted_avg_WMT_Price_Gap_after_Units'),
           (F.sum('cos_price_gap_before')/F.sum('predicted_biannual_units_Units')).alias('weighted_avg_COS_Price_Gap_before_Units'),
           (F.sum('cos_price_gap_after')/F.sum('predicted_units_Units')).alias('weighted_avg_COS_Price_Gap_after_Units'),
           (F.sum('bj_price_gap_before')/F.sum('predicted_biannual_units_Units')).alias('weighted_avg_BJ_Price_Gap_before_Units'),
           (F.sum('bj_price_gap_after')/F.sum('predicted_units_Units')).alias('weighted_avg_BJ_Price_Gap_after_Units'))
           
RUM_summary_pd['Current_price_gap_WMT'].insert(2, price_gaps_Units.groupBy("CBSA_Units").mean().collect()[0]['avg(weighted_avg_WMT_Price_Gap_before_Units)'])
RUM_summary_pd['Recommended_price_gap_WMT'].insert(2, price_gaps_Units.groupBy("CBSA_Units").mean().collect()[0]['avg(weighted_avg_WMT_Price_Gap_after_Units)'])

RUM_summary_pd['Current_price_gap_Costco'].insert(2, price_gaps_Units.groupBy("CBSA_Units").mean().collect()[0]['avg(weighted_avg_COS_Price_Gap_before_Units)'])
RUM_summary_pd['Recommended_price_gap_Costco'].insert(2, price_gaps_Units.groupBy("CBSA_Units").mean().collect()[0]['avg(weighted_avg_COS_Price_Gap_after_Units)'])

RUM_summary_pd['Current_price_gap_BJ'].insert(2, price_gaps_Units.groupBy("CBSA_Units").mean().collect()[0]['avg(weighted_avg_BJ_Price_Gap_before_Units)'])
RUM_summary_pd['Recommended_price_gap_BJ'].insert(2, price_gaps_Units.groupBy("CBSA_Units").mean().collect()[0]['avg(weighted_avg_BJ_Price_Gap_after_Units)'])

In [0]:
RUM_summary_pd

In [0]:
for key, value in RUM_summary_pd.items():
  if value == [None, None, None]:
    print(key)
    RUM_summary_pd[key] = ['Null', 'Null', 'Null']

RUM_summary_df = pd.DataFrame(RUM_summary_pd)
sdf = spark.createDataFrame(RUM_summary_df)
sdf\
    .write.format('delta').mode('overwrite').option('overwriteSchema', 'true')\
    .saveAsTable(dbname + '.Summary_analysis_' + budget + 'M_budget_zone' + zone_id)

display(sdf)

In [0]:
# display(spark.read.table('s0a02qd.summary_rum_analysis_mes_moo_units_price_investment_nj_1_5m_biannual_sept_'))